In [28]:
! pip install pyspark


In [29]:
from pyspark.sql import SparkSession

spark= SparkSession.builder.appName("ml_project").getOrCreate()

In [30]:
spark

In [31]:
! git clone https://github.com/education454/admission_dataset

fatal: destination path 'admission_dataset' already exists and is not an empty directory.


In [32]:
df = spark.read.csv('admission_dataset/Admission_Predict_Ver1.1.csv', header=True, inferSchema=True)

In [33]:
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [34]:
shape = (df.count(), len(df.columns))

print(shape)

(500, 9)


In [35]:
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [36]:
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

In [37]:
df= df.drop('Serial No')

In [38]:
df.show(2)

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
+---------+-----------+-----------------+---+---+----+--------+---------------+
only showing top 2 rows



In [39]:
print(df[df['GRE Score'].isNull()].count())

0


In [40]:
for col in df.columns:
  print(col + ':' , df[df[col].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


In [41]:
print(df.stat.corr('GRE Score', 'Chance of Admit'))

0.8103506354632601


In [42]:
for col in df.columns:
  print(f"{col} is {round(df.stat.corr(col, 'Chance of Admit'),3)} correlated with the target variable Chance of Admit")

GRE Score is 0.81 correlated with the target variable Chance of Admit
TOEFL Score is 0.792 correlated with the target variable Chance of Admit
University Rating is 0.69 correlated with the target variable Chance of Admit
SOP is 0.684 correlated with the target variable Chance of Admit
LOR is 0.645 correlated with the target variable Chance of Admit
CGPA is 0.882 correlated with the target variable Chance of Admit
Research is 0.546 correlated with the target variable Chance of Admit
Chance of Admit is 1.0 correlated with the target variable Chance of Admit


In [43]:
from pyspark.ml.feature import VectorAssembler 

assembler = VectorAssembler(inputCols=['GRE Score','TOEFL Score', 'CGPA'],outputCol='features' )

In [44]:
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

In [45]:
from pyspark.ml.regression import LinearRegression

final_data = output_data.select('features', 'Chance of Admit')

In [46]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [47]:
train, test = final_data.randomSplit([0.7,0.3])


In [48]:
models = LinearRegression(featuresCol= 'features', labelCol='Chance of Admit')

model= models.fit(train)


In [49]:
print('Coefficients:', model.coefficients)

print('Intercept:', model.intercept)

Coefficients: [0.0025480840408148298,0.0037046175602137694,0.13852337128250372]
Intercept: -1.6711259012494226


In [50]:
summary = model.summary

In [51]:
print('RMSE :', summary.rootMeanSquaredError)

print('r2 :', summary.r2)

RMSE : 0.06232714264849722
r2 : 0.8020875053626523


In [52]:
predictions= model.transform(test)

In [53]:
predictions.show()

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|[290.0,100.0,7.56]|           0.47| 0.4855169135039832|
|  [293.0,97.0,7.8]|           0.64| 0.5152929220535869|
|  [295.0,93.0,7.2]|           0.46| 0.4224565971248593|
| [295.0,99.0,7.57]|           0.37| 0.4959379498606684|
|[295.0,101.0,7.86]|           0.69| 0.5435189626530219|
| [296.0,99.0,7.28]|           0.47|0.45831425622955724|
|[296.0,101.0,7.68]|            0.6|  0.521132839862986|
| [297.0,99.0,7.81]|           0.54|  0.534279727050099|
| [298.0,99.0,7.46]|           0.53|0.48834463114203785|
| [299.0,96.0,7.86]|           0.54| 0.5351882110152126|
|[299.0,100.0,7.42]|           0.42|0.48905639789176636|
|[299.0,100.0,7.88]|           0.51| 0.5527771486817175|
|[299.0,100.0,7.88]|           0.68| 0.5527771486817175|
|[299.0,102.0,8.62]|           0.56| 0.6626936785511979|
| [300.0,98.0,8.02]|           

In [54]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='Chance of Admit', metricName='r2')

print('r2 score:', evaluator.evaluate(predictions))

r2 score: 0.8082328760862674
